In [7]:
import tensorflow as tf
import numpy as np
import pandas as pd
import sys
import matplotlib.pyplot as plt
import os

In [8]:
arr = os.listdir('.')
print(arr)

['Untitled1.ipynb', 'FinalDBelon.csv', 'Data_train.csv', 'DataNONEL_clean.csv', 'Untitled.ipynb', 'Clean_Tweets_EM.ipynb', 'README.md', 'Tweets_Not_ElonMusk.csv', 'DataBase.csv', '.gitignore', 'DataElon_clean.csv', 'Vectorisation.ipynb', '.ipynb_checkpoints', 'Label_train.csv', 'Clean_Tweets_NONEM.ipynb', '.git', 'TweetsElonMusk.csv']


In [9]:
assert hasattr(tf, 'function')

In [10]:
Data_train = pd.read_csv("Data_train.csv")
Label_train = pd.read_csv("Label_train.csv")

In [86]:
Data_train.shape

print(Label_train.loc[0, 'label'])

1


In [202]:
model = tf.keras.models.Sequential()
#model.add(tf.keras.layers.Flatten(input_shape=[1, 961]))

model.add(tf.keras.layers.Dense(256, activation="relu"))
model.add(tf.keras.layers.Dense(128, activation="relu"))
model.add(tf.keras.layers.Dense(2, activation="softmax"))

# La prediction : model_output = model.predict(Tweet)

In [163]:
Tweet_1 = Data_train.values
Tweet_1.shape

(11130, 961)

In [196]:
Tweet_2 = Tweet_1.astype(dtype='float32')

In [211]:
Label_1 = Label_train.values
Label_1.shape

(11130, 1)

In [228]:
Label_2 = Label_1[:,0].astype(dtype='uint8')

In [231]:
model.predict(Tweet_2)

array([[0.493116  , 0.5068841 ],
       [0.48549724, 0.5145028 ],
       [0.4950072 , 0.5049928 ],
       ...,
       [0.46324983, 0.53675014],
       [0.48198888, 0.51801115],
       [0.47764513, 0.5223549 ]], dtype=float32)

In [232]:
model.summary()

Model: "sequential_22"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_38 (Dense)            (None, 256)               246272    
                                                                 
 dense_39 (Dense)            (None, 128)               32896     
                                                                 
 dense_40 (Dense)            (None, 2)                 258       
                                                                 
Total params: 279,426
Trainable params: 279,426
Non-trainable params: 0
_________________________________________________________________


In [234]:
model.compile(loss="sparse_categorical_crossentropy", optimizer="sgd", metrics=["accuracy"])

In [236]:
model.fit(Tweet_2, Label_2, epochs=15)

Epoch 1/15
348/348 [==============================] - 0s 1ms/step - loss: 0.3225 - accuracy: 0.8606
Epoch 2/15
348/348 [==============================] - 0s 1ms/step - loss: 0.3086 - accuracy: 0.8677
Epoch 3/15
348/348 [==============================] - 0s 1ms/step - loss: 0.2961 - accuracy: 0.8727
Epoch 4/15
348/348 [==============================] - 0s 1ms/step - loss: 0.2867 - accuracy: 0.8774
Epoch 5/15
348/348 [==============================] - 0s 1ms/step - loss: 0.2776 - accuracy: 0.8803
Epoch 6/15
348/348 [==============================] - 0s 1ms/step - loss: 0.2695 - accuracy: 0.8832
Epoch 7/15
348/348 [==============================] - 0s 1ms/step - loss: 0.2627 - accuracy: 0.8867
Epoch 8/15
348/348 [==============================] - 0s 1ms/step - loss: 0.2568 - accuracy: 0.8902
Epoch 9/15
348/348 [==============================] - 0s 1ms/step - loss: 0.2509 - accuracy: 0.8926
Epoch 10/15
348/348 [==============================] - 0s 1ms/step - loss: 0.2464 - accuracy: 0.8943

In [249]:
model.predict(Tweet_2[7000:7001])

array([[0.98605657, 0.01394342]], dtype=float32)

In [243]:
Data_train.columns

Index(['federal', 'engineers', 'sea', 'going', 'doesnt', 'purposetour',
       'america', 'model', 'health', 'boca',
       ...
       'countries', 'evafoxu', 'welcome', 'boards', 'join', 'la', 'guys',
       'guess', 'taylorlurking', 'are'],
      dtype='object', length=961)

In [251]:
T = "hello my name is   elon"
Texte = T.split(" ")
Texte

['hello', 'my', 'name', 'is', '', '', 'elon']

In [252]:
for index, value in enumerate(Texte):
    print("Index = ", index, " Value = ", value)

Index =  0  Value =  hello
Index =  1  Value =  my
Index =  2  Value =  name
Index =  3  Value =  is
Index =  4  Value =  
Index =  5  Value =  
Index =  6  Value =  elon


In [ ]:
def Vectorisation(Data_train, tweet):   #prend un tweet en argument sous la forme d'une string et renvoi son vecteur sous forme d'une database dans l espace vectoriel de la matrice
    Vecteurs = Data_train.columns
    row = []
    
    dictionnary = dict()
    for mot in Vecteurs:
        dictionnary[mot] = 0
    
    Mot_tweet = tweet.split(" ")
    
    for mot in Mot_tweet::
            
    
    
    

In [253]:
int(True)

1